# SQL à la main avec la DB API

On va utiliser ici SQLite car :
- le support de SQLite est inclus en standard dans Python : pas de driver à installer
- c'est une base de données « embarquée » : pas de serveur de bases de données à lancer


In [27]:
import sqlite3

## Établir une connexion

In [28]:
#!del test.db

In [29]:
connection = sqlite3.connect("test.db")

In [30]:
print(connection)

In [31]:
import os
assert os.path.exists("test.db")

## Exécuter des commandes

In [32]:
connection.execute(
    """
    CREATE TABLE IF NOT EXISTS fruits (
        id integer PRIMARY KEY,
        nom text NOT NULL
    );
    """
)
connection.execute("INSERT INTO fruits (nom) VALUES ('banane');")
connection.execute("INSERT INTO fruits (nom) VALUES ('poire');")
connection.execute("INSERT INTO fruits (nom) VALUES ('framboise');")

## Exécuter des requêtes et examiner les résultats

Une requête renvoit un « curseur » :

In [33]:
cursor = connection.execute("SELECT * FROM fruits;")
cursor

Le curseur permet de récupérer les résultats sous forme de tuples :

In [34]:
for row in cursor.fetchall():
    print(row)

(1, 'banane')
(2, 'poire')
(3, 'framboise')
(4, 'banane')
(5, 'poire')
(6, 'framboise')
(7, 'pomme de reinette')
(8, "pomme d'api")
(9, 'banane')
(10, 'poire')
(11, 'framboise')


## Requêtes paramétrées

In [35]:
for nom in ["pomme de reinette", "pomme d'api"]:
    sql = "INSERT INTO fruits (nom) VALUES ('{}');".format(nom)
    print(sql)
    connection.execute(sql)  # mauvaise idée !

INSERT INTO fruits (nom) VALUES ('pomme de reinette');
INSERT INTO fruits (nom) VALUES ('pomme d'api');


OperationalError: near "api": syntax error

In [38]:
for nom in ["pomme d'api"]:
    connection.execute(
        "INSERT INTO fruits (nom) VALUES (?);",
        (nom,)
    )  # laisser le driver injecter de manière sûre
connection.execute("COMMIT;")

In [43]:
connection.executemany(
    "INSERT INTO fruits (nom) VALUES (?);",
    [
        ("ananas",),
        ("clémentine",),
    ]
)
connection.commit()

In [44]:
cursor = connection.execute("SELECT * FROM fruits;")
cursor.fetchall()

[(1, 'banane'),
 (2, 'poire'),
 (3, 'framboise'),
 (4, 'banane'),
 (5, 'poire'),
 (6, 'framboise'),
 (7, 'pomme de reinette'),
 (8, "pomme d'api"),
 (9, 'banane'),
 (10, 'poire'),
 (11, 'framboise'),
 (12, 'pomme de reinette'),
 (13, "pomme d'api"),
 (14, "pomme d'api"),
 (15, 'ananas'),
 (16, 'clémentine'),
 (17, 'ananas'),
 (18, 'clémentine')]

## Fermer la connexion

In [25]:
connection.close()